# Обучение модели 

In [4]:
# Подключаем библиотеки

In [1]:
# Библиотека, которая позволяет создавать модели нейронных сетей 
import tensorflow as tf
from sklearn.model_selection import train_test_split
# Позволяет создать массив фреймов аудиопотока 
import numpy as np
import os
# Связанные с numpy
from IPython.display import Audio
from IPython.core.display import HTML
# Библиотека, которая позволяет рисовать спектрограмму частоты звука от времени
# Работает с преобразованием Фурье 
from scipy.signal import spectrogram

In [2]:
# Все еще подключаем библиотеки

In [8]:
# Для парсинга wav-файлов 
import scipy.io.wavfile as wav
# Мел частотные кепстральные коэффициенты (спектрограмма с мел-шкалой вместо оси у)
# Строит характеристику частот от времени в сэмплах через STFT 
from python_speech_features import fbank, mfcc
import time
# LSTM - для вызова рекурсивной функции
# Dense - для связи всех нейронов предыдущего слоя с текущим (выходным) слоем
# Convolution - сверта слоев
from keras.layers import LSTM, Dense, Convolution1D
# Позволяет давать послойное описание модели 
from keras.models import Sequential
# TimeDistributed - Один из способов работы с Dense 
# Bidirectional - получение информации не только от прошлого и самого себя, но от будущего  
from keras.layers.wrappers import TimeDistributed, Bidirectional
# Последовательность
from itertools import zip_longest
# Работа с графиками в jupyter 
%matplotlib inline

In [9]:
# Функция-декодер, принимает train_decoded и inv_mapping(?), возвращает(?) 
# что в ней происходит

In [65]:
# d - train_decoded 
# mapping - inv_mapping
def decode(d, mapping):
# Определяет 1-D тензор 
    shape = d.dense_shape
# batch_size определяет количество сэмплов, на которых учится НС  
# В нашем случае НС будет учиться на выборке размера batch_size num_epochos (700) раз 
    batch_size = shape[0]
# np.zeros возвращает новый тензор заданной формы и типа,
# заполненный нулями
    ans = np.zeros(shape=shape, dtype=int)
# np.int можно сравнить с сишным лонгом
# последовательнотсь длин батчей и дозаполнение нулями
    seq_lengths = np.zeros(shape=(batch_size, ), dtype=np.int)
    
    
# Пробегаемся по запакованным индексам и весам каждый индекс и вес          
    for ind, val in zip(d.indices, d.values):
# Берем индексы 0 и 1 второго слоя тензора SparseTensorValue train_decoded
# и элементу с этими индексами в 2-D нулевом массиве присваем значение
# val из train_decoded.values - 1-D вложенного тензора  
        ans[ind[0], ind[1]] = val
# очередь длин - берем максимум из  соседнего с seq_lengths[ind[0]] элемента 
# ind[0] = train_decoded.indices[0], т.к. indices - 2-D тензор 
        el = seq_lengths[ind[0]] 
        el = max(el, ind[1] + 1)
    ret = []
    
    for i in range(batch_size):
# Принимает массив         
        ret.append("".join(map(lambda s: mapping[s], ans[i, :seq_lengths[i]])))
    return ret


In [69]:
a = [1,2,3]
print(a[0] + 1)


2


In [64]:
# Функция, которая преобразует входящие данные 
# (список списков list_of_lists) к тензору  

In [42]:
def list_2d_to_sparse(list_of_lists):
# Преобразуем список списков к виду tf.SparseTensorValue(indices, values, shape)
# indices - 2-D тензор формы [N, ndims], который уточняет значения индексов для тензора с весами 
# (ненулевые индексы)
# values - 1-D тензор с данными любого типа и формой[N], который содержит значения весов для   
# каждого элемента с индексами из indices 
# shape - 1-D int64 тензор формы[ndims], который определяет форму тензора. Берет список, индексующий
# элементы в каждом измерении 
    indices, values = [], []
    for i, sublist in enumerate(list_of_lists):
        for j, value in enumerate(sublist):
            indices.append([i, j])
            values.append(value)
#   Привет, Кирилл. Сейчас 6 утра и следующие комменты тебя повеселят. Но мне нихрена не смешно :(
# max(map(len, list_of_lists)) - максимальная длина списка из списка списков
# dense_shape = [длина списка списков, максимальная длина списка из списка списков]     
    dense_shape = [len(list_of_lists), max(map(len, list_of_lists))]
# переводим данные в сишный лонг
    return tf.SparseTensorValue(indices = np.array(indices),
                                values = np.array(values),
                                dense_shape = np.array(dense_shape))

In [43]:
# Словарь для побуквенного распознавания

In [44]:
vocabulary = { 'а': 1,
               'б': 2,
               'в': 3,
               'г': 4,
               'д': 5,
               'е': 6,
               'ё': 7,
               'ж': 8,
               'з': 9,
               'и': 10,
               'й': 11,
               'к': 12,
               'л': 13,
               'м': 14,
               'н': 15,
               'о': 16,
               'п': 17,
               'р': 18,
               'с': 19,
               'т': 20,
               'у': 21,
               'ф': 22,
               'х': 23,
               'ц': 24,
               'ч': 25,
               'ш': 26,
               'щ': 27,
               'ъ': 28,
               'ы': 29,
               'ь': 30,
               'э': 31,
               'ю': 32,
               'я': 33
             }
# Преобразование в словарь 
inv_mapping = dict(zip(vocabulary.values(), vocabulary.keys()))
inv_mapping[34]='<пробел>'

In [45]:
# Добавление каталога аудио

In [46]:
# Каталог с базой аудио
voxforge_dir = './Voxforge'
# Папки в каталоге
speaker_folders = os.listdir(voxforge_dir)
speaker_folders1=[]
x=0
# Делаем массив директорий к аудиофайлам
for i in speaker_folders:
    speaker_folders1.append(i)
    x=x+1
    if x>75:
        break

In [47]:
# Получаем пути к аудиодорожкам

In [48]:
def list_files_for_speaker(speaker, folder):
# Создает список wav-файлов для данного "спикера" из дата-сета (voxforge)
# Список(подстрока, содержащая имя "спикера", имя папки)  
    speaker_folders = [d for d in os.listdir(folder) if speaker in d]
    wav_files = []
# Загрузка аудиофайлов из датасета
    for d in speaker_folders:
        for f in os.listdir(os.path.join(folder, d, 'wav')):
            wav_files.append(os.path.abspath(os.path.join(folder, d, 'wav', f)))
# Возвращает список путей к wav файлам     
    return wav_files

In [49]:
# Ищем отличительные признаки в сэмплах 

In [50]:
def extract_features_and_targets(wav_file):
# wav=[]
# Обработка текста
# Форматирование (?) wav -> txt
    txt_file = wav_file.replace('/wav/', '/txt/').replace('.wav', '.txt')
    try:
# txt_file - путь к файлу
# заливаем в буфер
        f = open(txt_file, 'rb')
# Ошибка ввода-вывода 
# Input Output Error
    except IOError as e:
        print("Не удалось открыть файл")
        return 0,0,0,0
    else:
        with f:
# Строка выше - все равно что with open(txt_file, 'rb') as f:
# Построчно читаем файл             
            for line in f.readlines():
                if line[0] == ';':
                    continue

# Читаем только по словарю [а-я] и заменяем все знаки на пустоту
            original = ' '.join(str(line, 'utf-8').strip().lower().split(' ')).replace('.', '').replace("'", '').replace('-', '').replace(',','')
# Меняем пробелы на более длинные 
            targets = original.replace(' ', '  ')
# Разделяем слова через пробелы 
            targets = targets.split(' ')
# Объявляем строку и начинаем ее с пробела
            stroka=[34]
            for i in targets:
                i1=i.encode("UTF-8")
                for j in range(0,len(i1),2):
# Через вызов decode() получаем буквы и добавляем их в строку 
                    stroka.append(vocabulary.get(i1[j:j+2].decode("utf-8"),34))
                if stroka[-1] != 34:
# Закончили с буквами
# Добавляем в конец строки пробел
                    stroka.append(34)
# Обработка звука
            fs, audio = wav.read(wav_file)
# Ищем отличительные черты через мел-кепстральные коэффициенты и преобразование Фурье            
# Мел-шкала отражает главным образом высоту звука, от которой, в свою очередь, его частота. 
# Эта зависимость нелинейна, особенно при низких частотах.
# Различные звуки имеют различные частоты и, соответственно, по-разному отображаются
# на мел-шкале.


            features = mfcc(audio, samplerate=fs, lowfreq=50)
            mean_scale = np.mean(features, axis=0)
            std_scale = np.std(features, axis=0)
            
            features = (features - mean_scale[np.newaxis, :]) / std_scale[np.newaxis, :]
            seq_len = features.shape[0]
            return features, stroka, seq_len, original


In [51]:
# Подготавливаем данные для обучения - примерно 3Гб

In [52]:
X = []
y = []
for j in speaker_folders1:
    wav_files = list_files_for_speaker(j, voxforge_dir)
    for i in wav_files:
        #i=i.encode('utf-8')
        features, stroka, seq_len, original = extract_features_and_targets(i)
        if seq_len != 0:
            X.append(features)
            y.append(stroka)
            
        break
    
    

In [53]:
# Создаем генератор
def batch(X_train, y_train, batch_size):
# num_features = 13
# 13 элементов в каждом (453) вложенном элементе X_train - в каждом i из X_train[i]  
# по сути мы можем взять любой X_train[А] из 453 вариантов А и num_features не изменится 
    num_features = X_train[0].shape[1]
    print(num_features)
# Количество элементов в X_train-е (453)
    n = len(X_train)
# Создаем массив из последовательности 0..452 и перемешиваем его 
    perm = np.random.permutation(n)
# batch_size = 100
# n // batch_size = 4
 
# Если мы отсортируем perm по возрастанию - получим массив 0..452     
#     perm.sort()
# если perm отсортирован по возрастанию, то
# np.resize(perm, (n // batch_size, batch_size)) делает из  perm 2-D массив и разбивает его на 4
# последовательности элементов от 100*а до 100*а+99, где а - целое число от 0 до 3
# если мы не сортируем perm, то получим 2-D массив с 4 элементами, в которых лежат
# неповторяющиеся числа от 0 до 452 по 100 в каждом элементе
# Следующий цикл проходит 4 раза 
    for batch_ind in np.resize(perm, (n // batch_size, batch_size)):
# batch_ind - элемент из resize-нутого perm-а (тот самый элемент, который состоит из 100 неповторяющихся
# значений в диапазоне 0..452)
# Т.е. мы берем каждый из элементов perm-а с индексом batch_ind, и т.к. batch_ind - индекс 2-D массива,
# то он сам является массивом
        print(batch_ind)
# Следующий цикл проходит 100 раз
# Мы берем первые 400 значений из X_train и записываем их в 2-D X_batch
        for i in batch_ind:
            X_batch = X_train[i]            
#             for i in range(len(X_batch)):
#                 print(X_batch)
        for i in batch_ind:
            y_batch = y_train[i] 
# циклы выше - то же. что и строка ниже 
# X_batch, y_batch = [X_train[i] for i in batch_ind], [y_train[i] for i in batch_ind]        
# конструкция list(map(function, arg)), где function - функция, которая применяется 
# к каждому элементу arg - аргумента составного типа данных(в нашем случае 2-D массив),
# равнозначна следующему коду:
# old_lit = ['1' , '2' , '3']
# new_list = []
# for i in old_list:
#     new_list.append(int(i))
# но в нашем случае мы создаем новый список sequence_lengths из длин элементов 2-D массива
        sequence_lengths = list(map(len, X_batch))  
#         print(sequence_lengths)

        X_batch_padded = np.array(list(zip_longest(*X_batch, fillvalue=np.zeros(num_features)))).transpose([1, 0, 2])
# zip_longest() преобразует аргументы в значение типа zip
# list(zip), где zip - переменная типа zip, повзоляет посмотреть, что именно лежит в переменной 
# (в целом, лист обратен зипу) 

# С учетом данных выше эту строку можно разложить так:
# zipped_X_batches = zip_longest(*X_batch, fillvalue=np.zeros(num_features))
# list_to_array = list(zipped_X_batches)
# X_batch_padded = np.array(list_to_array)
 

# !!!
# зачем это происходит?

# возвращаем данные 
        yield X_batch_padded, sequence_lengths, list_2d_to_sparse(y_batch), y_batch

In [54]:
# Обучаемся в начале на коротких данных 
y_train1=[]
X_train1=[]
for i in range(len(y)):
    if len(y[i]) <75:
        y_train1.append(y[i])
        X_train1.append(X[i])

In [55]:
# X_batch, seq_lens_batch, y_batch, y_batch_orig=batch(X_train1, y_train1, 100)

In [56]:
# n = 13
# batch_size = 100
# perm = np.random.permutation(n)
# y_train1=[]
# X_train1=[]
# for i in range(len(y)):
#     if len(y[i]) < 75:
#         y_train1.append(y[i])
#         X_train1.append(X[i])
# print("S")
# for batch_ind in np.resize(perm, (4, batch_size)):
#     print(batch_ind.shape)
#     print(len(batch_ind))
#     print(batch_ind)
#     x_batch = [X_train1[i] for i in batch_ind]
#     # print("x_batch", i, x_batch)
#     y_batch = [y_train1[i] for i in batch_ind]
#     # print("y_batch", i, y_batch)
    

In [57]:
# perm = np.random.permutation(n) 
# print(perm)
# a = 100
# n = 13
# print(np.resize(perm, (n // a, a)))


In [58]:
# aa.resize((1,7))
# print(aa)

In [59]:
# X_batch, seq_lens_batch, y_batch, y_batch_orig=batch(X_train1, y_train1, 100)

<!-- # Модель нейронной сети -->

In [60]:
# Создание графа
graph = tf.Graph()
with graph.as_default():
    input_X = tf.placeholder(tf.float32, shape=[None, None, 13],name="input_X")
    labels = tf.sparse_placeholder(tf.int32)
    seq_lens = tf.placeholder(tf.int32, shape=[None],name="seq_lens")

# Sequential() 
    model = Sequential()
#     LSTM(units, activation = 'tanh', recurrent_activation= 'sigmoid', use_bias = true,
#          kernel_initializer = 'glorot_uniform', recurrent_initializer = 'orthogonal', 
#          bias_initializer = 'zeros', ... )
# Слои двунаправленных нейронов 
    model.add(Bidirectional(LSTM(128, return_sequences=True, implementation=2), input_shape=(None, 13)))
    model.add(Bidirectional(LSTM(128, return_sequences=True, implementation=2)))
# Выходной слой 
    model.add(TimeDistributed(Dense(len(inv_mapping) + 2)))
    
    final_seq_lens = seq_lens
    
#     Что происходит?? 
#     !!!!!!!
#     print(logits)
    print(model)
    logits = model(input_X)
# "logits" does not change so whyyyyyy
    logits = tf.transpose(logits, [1, 0, 2])
    print(logits)
    print(model)

    ctc_loss = tf.reduce_mean(tf.nn.ctc_loss(labels, logits, final_seq_lens))
    # ctc_greedy_decoder? merge_repeated=True
    decoded, log_prob = tf.nn.ctc_greedy_decoder(logits, final_seq_lens)
    ler = tf.reduce_mean(tf.edit_distance(tf.cast(decoded[0], tf.int32), labels))

    train_op = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(ctc_loss)

Tensor("transpose:0", shape=(?, ?, 36), dtype=float32)


In [61]:
num_epochs = 100
epoch_save_step=20

In [62]:
# Обучаемся в начале на коротких данных 
y_train1=[]
X_train1=[]
for i in range(len(y)):
    if len(y[i]) < 75:
        y_train1.append(y[i])
        X_train1.append(X[i])

Вот отсюда не чего не понятно, код написан на пионе втором , и тензор тоже неизвестно какой версии, я в принципе большую часть всего переделал под третий, а дальше не могу понять что происходит

In [63]:
# Произведение вычислительных процессов для частей графа 
# Бесконенчый цикл, пока выполняется условие
# Сессия позволяет хранить значения графа и производить с ними вычисления 
with tf.Session(graph = graph) as session:
    print("----------------------------------------------------")
# Храним все переменные модели
    saver = tf.train.Saver(tf.global_variables())
# Строка, кот. явл. частью названия файла
    snapshot = "ctc"
# Ищет имя последнего чекпоинт-файла
# Чекпоинт не содержит описание вычислений в модели
# Чекпоинт нужен, когда исходный код использует доступные (сохраненные) значения (веса и индексы) графа
# Чекпоинт хранит готовую модель на каком-то шаге 
# Если папка пустая, то чекпоинт хранит пустое значение 
    checkpoint = tf.train.latest_checkpoint(checkpoint_dir="checkpoint1")
# Определеям значение стартовой эпохи
# Количество эпох = количеству обучений на одном дата-сете
    last_epoch = 0

# Сохранение через каждое заданное кол-во эпох (в нашем случае шаг - 10)
# шаг epoch_save_step = 10
# После обучения на 10 эпохах делаем сохранение, иначе учимся дальше 
    if checkpoint:
# Произошло сохранение        
        print("[i] LOADING checkpoint " + checkpoint)
        try:
# Обновляем данные с предыдущего сохранения    
            saver.restore(session, checkpoint)
# Меняем имя файла на "..."+(n+1) 
            last_epoch = int(checkpoint.split('-')[-1]) + 1
            print("[i] start from epoch %d" % last_epoch)
        except:
# Если структура последнего чекпоинта неправильная
# Создаем новый чекпоинт              
            print("[!] incompatible checkpoint, restarting from 0")
    else:
# Если нет чекпоинтов (chekpoint = None) 
# Инициализирум веса и биасы
        tf.global_variables_initializer().run()


    for epoch in range(last_epoch, num_epochs):
# X_batch 
# Y_batch
# seq_lens_batch
# y_batch_orig
        for X_batch, seq_lens_batch, y_batch, y_batch_orig in batch(X_train1, y_train1, 100):
            feed_dict = {
                input_X: X_batch,
                labels: y_batch,

                seq_lens: seq_lens_batch
            }
            
            
# train_loss = session.run([ctc_loss])
# 
            train_loss, train_ler, train_decoded, true, _ = session.run([ctc_loss, ler, decoded[0], labels, train_op], feed_dict=feed_dict)
        print("train_decoded", train_loss)
        if epoch % epoch_save_step == 0 and epoch > 0:
                print("[i] SAVING snapshot %s" % snapshot)
#                 del tf.get_collection_ref ( ' LAYER_NAME_UIDS ' )[ 0 ]
                saver.save(session, "checkpoint1/" + snapshot + ".ckpt", epoch)

#         for X_batch, seq_lens_batch, y_batch, y_batch_orig in batch(X_test, y_test, 4):
#             feed_dict = {
#                 input_X: X_batch,
#                 labels: y_batch,
#                 seq_lens: seq_lens_batch
#             }
#             test_loss, test_ler, test_decoded, true = session.run([ctc_loss, ler, decoded[0], labels], feed_dict=feed_dict)
#         print(epoch, train_loss, train_ler)#,  test_loss, test_ler)
        ret = decode(train_decoded, inv_mapping)[:10]
        for i in range(len(ret)):
            print(str(ret[i])),
        print(time.ctime())
        decode1(y_batch_orig[0],inv_mapping)

----------------------------------------------------
[i] LOADING checkpoint checkpoint1/ctc.ckpt-10
INFO:tensorflow:Restoring parameters from checkpoint1/ctc.ckpt-10
[i] start from epoch 11
13


NameError: name 'train_loss' is not defined

In [ ]:
ret = decode(train_decoded, inv_mapping)[:10]

Обучаемся на средних данных (длина предложения меньше 181 символа)

Если у Вас мощный компьютер, можно обучиться на полных данных

In [ ]:
# num_epochs=1000
# epoch_save_step=5
# with tf.Session(graph=graph) as session:

#     saver = tf.train.Saver(tf.global_variables())
#     snapshot = "ctc"
#     checkpoint = tf.train.latest_checkpoint(checkpoint_dir="checkpoint1")
#     last_epoch = 0

#     if checkpoint:
#         print("[i] LOADING checkpoint " + checkpoint)
#         try:
#             saver.restore(session, checkpoint)
#             last_epoch = int(checkpoint.split('-')[-1]) + 1
#             print("[i] start from epoch %d" % last_epoch)
#         except:
#             print("[!] incompatible checkpoint, restarting from 0")
#     else:
#         # Initializate the weights and biases
#         tf.global_variables_initializer().run()


#     for epoch in range(last_epoch, num_epochs):
#         for X_batch, seq_lens_batch, y_batch, y_batch_orig in batch(X, y, 4):
#             feed_dict = {
#                 input_X: X_batch,
#                 labels: y_batch,

#                 seq_lens: seq_lens_batch
#             }
#             train_loss, train_ler, train_decoded, true, _ = session.run([ctc_loss, ler, decoded[0], labels, train_op], feed_dict=feed_dict)
#         if epoch % epoch_save_step == 0 and epoch > 0:
#                 print("[i] SAVING snapshot %s" % snapshot)
# #                 del tf.get_collection_ref ( ' LAYER_NAME_UIDS ' )[ 0 ]
#                 saver.save(session, "checkpoint1/" + snapshot + ".ckpt", epoch)

# #         for X_batch, seq_lens_batch, y_batch, y_batch_orig in batch(X_test, y_test, 4):
# #             feed_dict = {
# #                 input_X: X_batch,
# #                 labels: y_batch,
# #                 seq_lens: seq_lens_batch
# #             }
# #             test_loss, test_ler, test_decoded, true = session.run([ctc_loss, ler, decoded[0], labels], feed_dict=feed_dict)
#         print(epoch, train_loss, train_ler)#,  test_loss, test_ler)
#         ret=decode(train_decoded, inv_mapping)[:10]
#         for i in range(len(ret)):
#             print(str(ret[i])),
#         print(time.ctime())
#         decode1(y_batch_orig[0],inv_mapping)